In [1]:
import time
#ini=time.time()
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
test=pd.read_csv('~/git/BD/temp/test.csv')
test.head(1)

,oc,oc_test,date,md_c,l_min_y,indv,l_max_y,ad_col,CO_MES,nick,KG_LIQUIDO,KG_LIQUIDO_test,md_r,CO_ANO
0,0,1,2012-07-01,1,15,0,15,0,7,MAP_k,NaN,NaN,NaN,2012


In [3]:
scores=pd.DataFrame()
for mes in test.CO_MES.unique():
    for n in test.nick.unique():
        for mdc in test.md_c.unique():
            for mdr in test.md_r.unique():
                for ad in test.ad_col.unique():
                    for indv in test.indv.unique():
                        test_t=test[
                        (test.CO_MES==mes)&
                        (test.nick==n)&
                        (test.md_c==mdc)&
                        (test.md_r==mdr)&
                        (test.ad_col==ad)&
                        (test.indv==indv)]
                        ac=round(metrics.accuracy_score(test_t.oc,test_t.oc_test),2)
                        pr=round(metrics.precision_score(test_t.oc,test_t.oc_test),2)
                        rc=round(metrics.recall_score(test_t.oc,test_t.oc_test),2)
                        test_t=test_t[(test_t.oc==1)&(test_t.oc_test==1)]
                        if len(test_t)>0:
                            r2=metrics.r2_score(test_t.KG_LIQUIDO,test_t.KG_LIQUIDO_test)
                            mape=metrics.mean_absolute_percentage_error(test_t.KG_LIQUIDO,test_t.KG_LIQUIDO_test)
                        else:
                            r2=np.nan
                        q=pd.DataFrame({'CO_MES':[mes],'nick':[n],'md_c':[mdc],'md_r':[mdr],'accuracy':[ac],
                                        'precision':[pr],'recall':[rc],'r2':[r2],'ad_col':[ad],'indv':[indv]
                                       })
                        scores=pd.concat([q,scores])

scores.CO_MES=scores.CO_MES.astype(int)
scores=scores.sort_values(by=['CO_MES','nick'],ascending=False)
scores.to_csv('~/git/BD/temp/score.csv',index=False)
scores.head()

,CO_MES,nick,md_c,md_r,accuracy,precision,recall,r2,ad_col,indv
0,7,UREIA_k,1,1.0,1.0,1.0,1.0,-1.094329,1,0
0,7,UREIA_k,1,1.0,1.0,1.0,1.0,-1.051423,0,0
0,7,UREIA_k,1,2.0,1.0,1.0,1.0,-2.445572,1,0
0,7,UREIA_k,1,2.0,1.0,1.0,1.0,-2.174748,0,0
0,7,UREIA_k,1,3.0,1.0,1.0,1.0,-3.132322,1,0


In [4]:
top_scores=scores.groupby(by=['CO_MES','nick']).r2.max().reset_index().merge(scores)
top_scores.to_csv('~/git/BD/temp/top_scores.csv',index=False)
top_scores

,CO_MES,nick,r2,md_c,md_r,accuracy,precision,recall,ad_col,indv
0,7,MAP_k,-0.093928,1,13.0,1.0,1.0,1.0,1,0
1,7,MOP_k,-0.568927,1,13.0,1.0,1.0,1.0,1,0
2,7,SAM_k,0.652960,1,13.0,1.0,1.0,1.0,1,0
3,7,UREIA_k,-1.051423,1,1.0,1.0,1.0,1.0,0,0


In [5]:
r2_pos_class_pos=scores[(scores.r2>0)&(scores.accuracy>.95)&(scores.precision>.95)&(scores.recall>.95)].reset_index(drop=True)

r2_pos_class_out=scores[(scores.r2>0)&((scores.accuracy<=.95)|(scores.precision<=.95)|(scores.recall<=.95))].reset_index(drop=True)

r2_out_class_pos=scores[(scores.r2<=0)&(scores.accuracy>.95)&(scores.precision>.95)&(scores.recall>.95)].reset_index(drop=True)

r2_out_class_out=scores[(scores.r2<=0)&((scores.accuracy<=.95)|(scores.precision<=.95)|(scores.recall<=.95))].reset_index(drop=True)

In [6]:
print('Aprovados: ',len(r2_pos_class_pos[['CO_MES','nick']].drop_duplicates()))

Aprovados:  1


In [7]:
aprovados=r2_pos_class_pos.groupby(by=['CO_MES','nick'])['r2'].max().reset_index().merge(r2_pos_class_pos)
aprovados.to_csv('~/git/BD/temp/aprovados.csv',index=False)
aprovados

,CO_MES,nick,r2,md_c,md_r,accuracy,precision,recall,ad_col,indv
0,7,SAM_k,0.65296,1,13.0,1.0,1.0,1.0,1,0


In [8]:
len(r2_pos_class_out[['CO_MES','nick']].drop_duplicates())

0

In [9]:
len(r2_out_class_pos[['CO_MES','nick']].drop_duplicates())

3

In [10]:
len(r2_out_class_out[['CO_MES','nick']].drop_duplicates())

0